# Identify trips out of LA

There are two main dataframe ‘Basecase’ and ‘trip_out_of_LA’. 
- ‘Basecase’ is the travel trajectory data that contains all travel information of each trip. 
- ‘trip_out_of_LA’ lists the trips that have the origin or destination locating outside LA county. 
   If the values of column ‘TAZ_Orig[i’ or  ‘TAZ_Dest[i’ are less than 5571, it means the location is out of LA.
   
Task: If the origin is out of LA, change ‘start_link’ in ‘Basecase’ to ‘0’; if the destination is out of LA then change ‘end_link’ in ‘Basecase’ to ‘0’.

In [15]:
import numpy as np
import pandas as pd

In [10]:
# Read Basecase into a pandas DataFrame
Base = pd.read_csv('Data\\Basecase.csv',delimiter=';')
pd.set_option('display.max_columns', None)
Base.head()

C:\Users\User\AppData\Local\Temp\ipykernel_21512\3062115749.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  Base = pd.read_csv('Data\\Basecase.csv',delimiter=';')


,person,trip_number,trip_id,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,main_mode,longest_distance_mode,modes,start_activity_type,end_activity_type,start_facility_id,start_link,start_x,start_y,end_facility_id,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop
0,2,1,2_1,06:41:30,00:12:50,00:00:00,11802,8762,NaN,car,car,Home,Work,NaN,400998002_2,1.948668e+06,578718.582238,NaN,398831767_0,1.940530e+06,581967.177854,NaN,NaN
1,2,2,2_2,18:01:50,00:22:19,00:00:00,12121,8792,NaN,car,car,Work,Home,NaN,398831767_0,1.940530e+06,581967.177854,NaN,400998002_2,1.948772e+06,578905.454005,NaN,NaN
2,3,1,3_1,10:13:00,00:37:35,00:00:00,13087,10653,NaN,car,car,Home,Shop,NaN,400997969_0,1.948184e+06,579096.609692,NaN,402514238_0,1.958522e+06,576523.834258,NaN,NaN
3,3,2,3_2,12:25:00,00:21:19,00:00:00,12990,10543,NaN,car,car,Shop,Home,NaN,402514238_0,1.958522e+06,576523.834258,NaN,400997969_0,1.948282e+06,579032.851227,NaN,NaN
4,4,1,4_1,09:48:34,04:52:56,00:00:00,193357,85577,NaN,car,car,Home,Shop,NaN,400998002_1,1.948787e+06,578718.324810,NaN,835412564_0,2.026958e+06,613543.368043,NaN,NaN


In [31]:
Base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4667765 entries, 0 to 4667764
Data columns (total 23 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   person                  object 
 1   trip_number             int64  
 2   trip_id                 object 
 3   dep_time                object 
 4   trav_time               object 
 5   wait_time               object 
 6   traveled_distance       int64  
 7   euclidean_distance      int64  
 8   main_mode               float64
 9   longest_distance_mode   object 
 10  modes                   object 
 11  start_activity_type     object 
 12  end_activity_type       object 
 13  start_facility_id       float64
 14  start_link              object 
 15  start_x                 float64
 16  start_y                 float64
 17  end_facility_id         float64
 18  end_link                object 
 19  end_x                   float64
 20  end_y                   float64
 21  first_pt_boarding_stop  float64

In [21]:
# Read Basecase into a pandas DataFrame
OutLA = pd.read_csv('Data\\trips_out_of_LA.csv')
OutLA.head()

,pid[i,tripId[i,TAZ_Orig[i,TAZ_Dest[i,Location out of LA (TAZ number <5571)
0,3,7,5910,3861,3861.0
1,3,8,3861,5910,3861.0
2,22,1,6020,3096,3095.0
3,22,2,3096,6020,NaN
4,24,9,6020,5504,NaN


In [14]:
OutLA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   pid[i                                  1048575 non-null  int64  
 1   tripId[i                               1048575 non-null  int64  
 2   TAZ_Orig[i                             1048575 non-null  int64  
 3   TAZ_Dest[i                             1048575 non-null  int64  
 4   Location out of LA (TAZ number <5571)  3 non-null        float64
 5   TAZ_Orig                               887097 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 48.0 MB


In [23]:
# Create 'trip_id' column by merging 'pid[i]' and 'tripId[i]'
OutLA['trip_id'] = OutLA['pid[i'].astype(str) + '_' + OutLA['tripId[i'].astype(str)
OutLA.head()

,pid[i,tripId[i,TAZ_Orig[i,TAZ_Dest[i,Location out of LA (TAZ number <5571),trip_id
0,3,7,5910,3861,3861.0,3_7
1,3,8,3861,5910,3861.0,3_8
2,22,1,6020,3096,3095.0,22_1
3,22,2,3096,6020,NaN,22_2
4,24,9,6020,5504,NaN,24_9


In [32]:
OutLA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   pid[i                                  1048575 non-null  int64  
 1   tripId[i                               1048575 non-null  int64  
 2   TAZ_Orig[i                             1048575 non-null  int64  
 3   TAZ_Dest[i                             1048575 non-null  int64  
 4   Location out of LA (TAZ number <5571)  3 non-null        float64
 5   trip_id                                1048575 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 48.0+ MB


In [58]:
# Perform a left join on 'trip_id'
merged_df = pd.merge(Base, OutLA[['TAZ_Orig[i','TAZ_Dest[i','trip_id']], on='trip_id', how='left')
merged_df.head()

,person,trip_number,trip_id,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,main_mode,longest_distance_mode,modes,start_activity_type,end_activity_type,start_facility_id,start_link,start_x,start_y,end_facility_id,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop,TAZ_Orig[i,TAZ_Dest[i
0,2,1,2_1,06:41:30,00:12:50,00:00:00,11802,8762,NaN,car,car,Home,Work,NaN,400998002_2,1.948668e+06,578718.582238,NaN,398831767_0,1.940530e+06,581967.177854,NaN,NaN,NaN,NaN
1,2,2,2_2,18:01:50,00:22:19,00:00:00,12121,8792,NaN,car,car,Work,Home,NaN,398831767_0,1.940530e+06,581967.177854,NaN,400998002_2,1.948772e+06,578905.454005,NaN,NaN,NaN,NaN
2,3,1,3_1,10:13:00,00:37:35,00:00:00,13087,10653,NaN,car,car,Home,Shop,NaN,400997969_0,1.948184e+06,579096.609692,NaN,402514238_0,1.958522e+06,576523.834258,NaN,NaN,NaN,NaN
3,3,2,3_2,12:25:00,00:21:19,00:00:00,12990,10543,NaN,car,car,Shop,Home,NaN,402514238_0,1.958522e+06,576523.834258,NaN,400997969_0,1.948282e+06,579032.851227,NaN,NaN,NaN,NaN
4,4,1,4_1,09:48:34,04:52:56,00:00:00,193357,85577,NaN,car,car,Home,Shop,NaN,400998002_1,1.948787e+06,578718.324810,NaN,835412564_0,2.026958e+06,613543.368043,NaN,NaN,NaN,NaN


In [59]:
# To check how many rows have values
print(merged_df['TAZ_Orig[i'].count())
print(merged_df['TAZ_Dest[i'].count())

8024
8024


In [60]:
# Replace values in 'start_link' column if 'TAZ_Orig[i]' < 5571
merged_df.loc[merged_df['TAZ_Orig[i'] < 5571, 'start_link'] = 0

# Replace values in 'end_link' column if 'TAZ_Orig[i]' < 5571
merged_df.loc[merged_df['TAZ_Dest[i'] < 5571, 'end_link'] = 0

In [61]:
# Filter rows where 'TAZ_Orig[i]' is 0 for checking
filtered_df = merged_df[merged_df['start_link'] == 0]
filtered_df.head()

,person,trip_number,trip_id,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,main_mode,longest_distance_mode,modes,start_activity_type,end_activity_type,start_facility_id,start_link,start_x,start_y,end_facility_id,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop,TAZ_Orig[i,TAZ_Dest[i
625,3368,1,3368_1,07:23:14,00:13:47,00:00:00,2728,504,NaN,car,car,Home,Maintenance,NaN,0,2.013437e+06,551129.723017,NaN,0,2.013162e+06,550706.674210,NaN,NaN,2005.0,1840.0
626,3368,2,3368_2,07:59:30,00:57:41,00:00:00,49691,30497,NaN,car,car,Maintenance,Discretionary,NaN,0,2.013162e+06,550706.674210,NaN,0,1.990726e+06,530049.985947,NaN,NaN,1840.0,2005.0
634,3369,3,3369_3,17:48:24,00:34:38,00:00:00,70815,50581,NaN,ride,ride,Discretionary,Home,NaN,0,1.967868e+06,542754.088299,NaN,664282771_1,2.017668e+06,551611.873363,NaN,NaN,2005.0,5779.0
1282,11863,1,11863_1,04:18:42,06:36:33,00:00:00,25116,17158,NaN,walk,walk,Home,Eatout,NaN,0,2.014852e+06,552377.415813,NaN,0,2.000177e+06,543486.232696,NaN,NaN,1758.0,1816.0
1283,11863,2,11863_2,10:55:15,07:04:02,00:00:00,26856,19183,NaN,walk,walk,Eatout,Home,NaN,0,2.000177e+06,543486.232696,NaN,0,2.017768e+06,551136.776869,NaN,NaN,1816.0,1758.0


In [63]:
# Export DataFrame to CSV
merged_df.to_csv('Data\\Basecase2.csv', index=False)